# 教學目標

- 複習 PTT 文章的爬蟲邏輯
- 熟悉單網站多網頁，透過列表將連結內的文章內容都爬下來

In [1]:
import requests
import re
import json
from urllib.parse import urljoin
from bs4 import BeautifulSoup

# PTT 八卦版網址
PTT_URL = 'https://www.ptt.cc/bbs/Gossiping/index.html'

## 範例重點

這個練習題在 PTT 單頁文章上的程式碼比較多，建議可以先把他寫成一個 function 方便後面程式邏輯的撰寫

In [2]:
def crawl_article(url):
    response = requests.get(url, cookies={'over18': '1'})
    
    # 假設網頁回應不是 200 OK 的話, 我們視為傳送請求失敗
    if response.status_code != 200:
        print('Error - {} is not available to access'.format(url))
        return
    
    # 將網頁回應的 HTML 傳入 BeautifulSoup 解析器, 方便我們根據標籤 (tag) 資訊去過濾尋找
    soup = BeautifulSoup(response.text)
    
    # 取得文章內容主體
    main_content = soup.find(id='main-content')
    
    # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
    metas = main_content.select('div.article-metaline')
    author = ''
    title = ''
    date = ''
    if metas:
        if metas[0].select('span.article-meta-value')[0]:
            author = metas[0].select('span.article-meta-value')[0].string
        if metas[1].select('span.article-meta-value')[0]:
            title = metas[1].select('span.article-meta-value')[0].string
        if metas[2].select('span.article-meta-value')[0]:
            date = metas[2].select('span.article-meta-value')[0].string

        # 從 main_content 中移除 meta 資訊（author, title, date 與其他看板資訊）
        #
        # .extract() 方法可以參考官方文件
        #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
        for m in metas:
            m.extract()
        for m in main_content.select('div.article-metaline-right'):
            m.extract()
    
    # 取得留言區主體
    pushes = main_content.find_all('div', class_='push')
    for p in pushes:
        p.extract()
    
    # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
    # 透過 regular expression 取得 IP
    # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
    try:
        ip = main_content.find(text=re.compile(u'※ 發信站:'))
        ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
    except Exception as e:
        ip = ''
    
    # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
    # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
    #
    # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
    #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
    filtered = []
    for v in main_content.stripped_strings:
        # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
        if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
            filtered.append(v)

    # 定義一些特殊符號與全形符號的過濾器
    expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
    for i in range(len(filtered)):
        filtered[i] = re.sub(expr, '', filtered[i])
    
    # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
    filtered = [i for i in filtered if i]
    content = ' '.join(filtered)
    
    # 處理留言區
    # p 計算推文數量
    # b 計算噓文數量
    # n 計算箭頭數量
    p, b, n = 0, 0, 0
    messages = []
    for push in pushes:
        # 假如留言段落沒有 push-tag 就跳過
        if not push.find('span', 'push-tag'):
            continue
        
        # 過濾額外空白與換行符號
        # push_tag 判斷是推文, 箭頭還是噓文
        # push_userid 判斷留言的人是誰
        # push_content 判斷留言內容
        # push_ipdatetime 判斷留言日期時間
        push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
        push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
        push_content = push.find('span', 'push-content').strings
        push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
        push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

        # 整理打包留言的資訊, 並統計推噓文數量
        messages.append({
            'push_tag': push_tag,
            'push_userid': push_userid,
            'push_content': push_content,
            'push_ipdatetime': push_ipdatetime})
        if push_tag == u'推':
            p += 1
        elif push_tag == u'噓':
            b += 1
        else:
            n += 1
    
    # 統計推噓文
    # count 為推噓文相抵看這篇文章推文還是噓文比較多
    # all 為總共留言數量 
    message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}
    
    # 整理文章資訊
    data = {
        'url': url,
        'article_author': author,
        'article_title': title,
        'article_date': date,
        'article_content': content,
        'ip': ip,
        'message_count': message_count,
        'messages': messages
    }
    return data

## 範例重點

分析列表取得要爬蟲的網址

In [3]:
# 對文章列表送出請求並取得列表主體
resp = requests.get(PTT_URL, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)
main_list = soup.find('div', class_='bbs-screen')
all_data = []

# 依序檢查文章列表中的 tag, 遇到分隔線就結束, 忽略這之後的文章
for div in main_list.findChildren('div', recursive=False):
    class_name = div.attrs['class']
    
    # 遇到分隔線要處理的情況
    if class_name and 'r-list-sep' in class_name:
        print('Reach the last article')
        break
    # 遇到目標文章
    if class_name and 'r-ent' in class_name:
        div_title = div.find('div', class_='title')
        a_title = div_title.find('a', href=True)
        article_URL = urljoin(PTT_URL, a_title['href'])
        article_title = a_title.text
        print('Parse {} - {}'.format(article_title, article_URL))
        
        # 呼叫上面寫好的 function 來對文章進行爬蟲
        parse_data = crawl_article(article_URL)
        
        # 將爬完的資料儲存
        all_data.append(parse_data)

Parse [問卦] 中共現在動武 台灣能擋嗎？ - https://www.ptt.cc/bbs/Gossiping/M.1580830677.A.4D3.html
Parse [問卦] 中國的罩門就是生化戰吧？ - https://www.ptt.cc/bbs/Gossiping/M.1580830687.A.F95.html
Parse [問卦] 球友要去見老爸怎辦？ - https://www.ptt.cc/bbs/Gossiping/M.1580830733.A.821.html
Parse Re: [新聞] 新北居家檢疫失聯3人 市府動員找人 - https://www.ptt.cc/bbs/Gossiping/M.1580830744.A.191.html
Parse [新聞] 抗疫殉職 湖南一28歲醫生勞累過度猝死 - https://www.ptt.cc/bbs/Gossiping/M.1580830756.A.4ED.html
Parse [問卦] 人資真的會去搜應徵者的社群？ - https://www.ptt.cc/bbs/Gossiping/M.1580830778.A.64A.html
Parse [爆卦] 林靜儀呼籲：一般日常生活不用戴口罩！ - https://www.ptt.cc/bbs/Gossiping/M.1580830785.A.4AA.html
Parse [新聞] 武漢首例確診台商「金先生」盼回台診治 - https://www.ptt.cc/bbs/Gossiping/M.1580830803.A.726.html
Parse [問卦] 新聞的疫情地圖 - https://www.ptt.cc/bbs/Gossiping/M.1580830931.A.7DD.html
Parse [新聞] WHO代表:健康人戴口罩 武漢肺炎風險更高 - https://www.ptt.cc/bbs/Gossiping/M.1580830945.A.ADE.html
Parse Re: [問卦] 台灣又沒社區傳染 到底幹嘛戴口罩呀 - https://www.ptt.cc/bbs/Gossiping/M.1580830978.A.CB1.html
Parse [問卦] 有一台剛到期三年focus二手車該不該衝? - https://ww

In [4]:
# 將爬完的資訊存成 json 檔案
with open('parse_data.json', 'w+') as f:
    json.dump(all_data, f, ensure_ascii=False, indent=4)